In [1]:
import pandas as pd
import gzip
import os
import matplotlib.pyplot as plt

GTEX_path = './GTEX/'
Xenium_path = './XeniumPanels/'

In [2]:
GTEX_dir = os.listdir(GTEX_path)
GTEX_files = [file for file in GTEX_dir if file.endswith('.gct.gz')]
Xenium_dir = os.listdir(Xenium_path)
Xenium_files = [file for file in Xenium_dir if file.endswith('.csv')]

In [3]:
print(Xenium_files)
Xenium_titles = Xenium_files.copy()
for i in range(len(Xenium_titles)):
    Xenium_titles[i] = Xenium_titles[i].split('_')[1]
print(Xenium_titles)

['Xenium_hSkin_preview_metadata.csv', 'Xenium_hBrain_v1_metadata.csv', 'Xenium_hColon_preview_metadata.csv', 'Xenium_hBreast_v1_metadata.csv', 'Xenium_hLung_v1_metadata.csv', 'Xenium_hMulti_v1_metadata.csv']
['hSkin', 'hBrain', 'hColon', 'hBreast', 'hLung', 'hMulti']


In [4]:
for x in range(len(Xenium_files)):
    
    gene_list_path = Xenium_path+Xenium_files[x]
    df_genes = pd.read_csv(gene_list_path)

    GTEX_names = GTEX_files.copy()
    for i in range(len(GTEX_names)):
        GTEX_names[i] = '_'.join(GTEX_names[i].split('_')[4:]).replace('.gct.gz', '')

    df_csv = df_genes[['Ensembl_ID', 'Gene']]
    df_csv.set_index('Ensembl_ID', inplace=True)
    for name in GTEX_names:
        df_csv[name] = 'MISSING'
            
    for g in range(len(GTEX_files)):
        # open and read the gzipped file
        with gzip.open(GTEX_path+GTEX_files[g], 'rt') as file:
            # skip the first 2 rows (header information) and read the data into a DataFrame
            df = pd.read_csv(file, sep='\t', skiprows=2, index_col=0)

        ensemble_ids = df_genes['Ensembl_ID'].unique()
        df['Name'] = df['Name'].str.split('.').str[0]
        filtered_df = df[df['Name'].isin(ensemble_ids)]
        filtered_df.set_index('Name', inplace=True)
        filtered_df = filtered_df.groupby('Name').sum()

        columns_to_average = filtered_df.columns.difference(['Description'])
        averages = filtered_df[columns_to_average].mean(axis=1)

        for index, row in df_csv.iterrows():
            if index in averages.index:
                # if averages[index] > 1:
                #     df_csv.at[index, GTEX_names[g]] = True
                # else:
                #     df_csv.at[index, GTEX_names[g]] = False
                df_csv.at[index, GTEX_names[g]] = averages[index]
    
    df_csv.to_csv('./GTEX_'+Xenium_titles[x]+'_count.csv', index=True)


/tmp/ipykernel_2948914/1519757112.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv[name] = 'MISSING'
/tmp/ipykernel_2948914/1519757112.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv[name] = 'MISSING'
/tmp/ipykernel_2948914/1519757112.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde